# 모델 검사, 모니터링 기법

## Callback

특정 epoch로 훈련을 하다가 overfitting 이 발생하는 지점을 찾고 해당 epoch으로 훈련하는 것 = 낭비

검증 손실이 향상되지 않을 때 훈련을 멈춘다

콜백을 사용하는 경우  
- 모델의 checkpoint 저장 : 모델의 현재 가중치를 저장  
- early stopping : 더 이상 향상되지 않을 때 훈련 중지  
- hyper-parameter 를 동적으로 조정 : 학습률 같은 것을 조정
- 훈련, 검증 지표를 로그에 기록 or 표현이 업데이트 될 때마다 시각화 : keras progress bar 도 하나의 콜백  

### ModelCheckpoint 콜백

훈련 동안 모델을 저장할 때 사용  
지금까지 가장 좋은 모델만 저장할 수 있음  

### EarlyStopping 콜백

정해진 에포크 동안 모니터링 지표가 향상되지 않으면 훈련을 중지  
overfitting 발생하면 훈련 중지

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks_list = [
    # patience : 1 epoch 보다 길게 = 에포크 2 이상 동안 monitor 지표가 향상되지 않으면 훈련 중지
    EarlyStopping(monitor='val_acc', patience=1),
    # val_loss 가 가장 좋은 모델만 저장
    ModelCheckpoint(filepath='my_model.h5', monitor='val_loss', save_best_only=True)
]

model.fit(x, y, 
          epochs=10, 
          batch_size=32, 
          callbacks=callbacks_list, # callback list 전달
          validation_data=(x_val, y_val))

### ReduceLROnPlateau 콜백

검증 손실이 향상되지 않을 때 학습률을 작게 조정  
지역 최솟값에서 빠져나올 수 있음  

In [ ]:
from keras.callbacks import ReduceLROnPlateau

callbacks_list = [
    # val_loss 를 모니터링
    # 검증 손실값이 10에포크 동안 좋아지지 않으면 콜백 호출
    # learning rate 에 0.1 을 곱함 = 학습률을 10배로 줄인다
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
]

model.fit(x, y, 
          epochs=10, 
          batch_size=32, 
          callbacks=callbacks_list, # callback list 전달
          validation_data=(x_val, y_val))

### 콜백 만들기

keras.callbacks.Callback을 상속받아 구현  

약속된 메서드를 구현

In [ ]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    # 훈련 모델에 대한 정보를 전달하기 위해 훈련 전에 호출됨
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.model.Model(model.input, layer_ouputs)  # 각 층의 활성화 출력을 반환
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data')
        
        # 검증 데이터의 첫번째 샘플
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activation_at_epoch_' + str(epoch) + '.npz', 'wb')
        np.savez(f, activations)
        f.close()

## Tensor Board

브라우저 기반의 시각화 도구  
텐서플로 백엔드로 케라스를 설정한 경우에만 사용 가능  

목적 : 훈련 모델의 내부에서 일어나는 모든 것을 시각적으로 모니터링  

In [9]:
import keras
from keras import layers, Input
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Model

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

imdb_input = Input(shape=(None,), dtype='int32', name='imdb')
embedding = layers.Embedding(max_features, 128, input_length=max_len, name='embed')(imdb_input)
x = layers.Conv1D(32, 7, activation='relu')(embedding)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(32, 7, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)

output = layers.Dense(1)(x)

model = Model([imdb_input], output)

model.summary()

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['acc'])

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/nahyeonan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/nahyeonan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
imdb (InputLayer)            [(None, None)]            0         
_________________________________________________________________
embed (Embedding)            (None, None, 128)         256000    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 32)          28704     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, None, 32)          7200      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33  

In [7]:
seq_model = keras.models.Sequential()
seq_model.add(layers.Embedding(max_features, 128, 
                               input_length=max_len, 
                               name='embed'))
seq_model.add(layers.Conv1D(32, 7, activation='relu'))
seq_model.add(layers.MaxPooling1D(5))
seq_model.add(layers.Conv1D(32, 7, activation='relu'))
seq_model.add(layers.GlobalMaxPooling1D())
seq_model.add(layers.Dense(1))

seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
__________________________________________________

텐서보드 로그 파일을 기록하는 디렉터리 생성

In [10]:
!mkdir my_log_dir

TensorBoard Callback  
지정된 디스크 위치에 로그 이벤트를 기록  

In [11]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',  # 로그파일 저장 위치
        histogram_freq=1,  # 1 에포크마다 활성화 출력의 히스토그램 기록
        embeddings_freq=1  # 1 에포크마다 임베딩 데이터 기록
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='my_model.h5', 
        monitor='val_loss', 
        save_best_only=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.1, 
        patience=4
    )
]

In [12]:
history = model.fit(x_train, y_train, 
                    epochs=20, 
                    batch_size=128, 
                    validation_split=0.2, 
                    callbacks=callbacks, 
                    verbose=2)

Epoch 1/20
157/157 - 18s - loss: 0.6381 - acc: 0.6913 - val_loss: 0.4696 - val_acc: 0.8168
Epoch 2/20
157/157 - 18s - loss: 0.4465 - acc: 0.8561 - val_loss: 0.7135 - val_acc: 0.8098
Epoch 3/20
157/157 - 18s - loss: 0.3680 - acc: 0.8860 - val_loss: 0.4721 - val_acc: 0.8630
Epoch 4/20
157/157 - 18s - loss: 0.3380 - acc: 0.9041 - val_loss: 0.8281 - val_acc: 0.8194
Epoch 5/20
157/157 - 18s - loss: 0.2935 - acc: 0.9229 - val_loss: 0.6939 - val_acc: 0.8426
Epoch 6/20
157/157 - 18s - loss: 0.2526 - acc: 0.9420 - val_loss: 0.6029 - val_acc: 0.8694
Epoch 7/20
157/157 - 18s - loss: 0.2060 - acc: 0.9568 - val_loss: 0.7363 - val_acc: 0.8606
Epoch 8/20
157/157 - 18s - loss: 0.1811 - acc: 0.9709 - val_loss: 0.8476 - val_acc: 0.8672
Epoch 9/20
157/157 - 17s - loss: 0.1553 - acc: 0.9795 - val_loss: 0.8411 - val_acc: 0.8656
Epoch 10/20
157/157 - 17s - loss: 0.1270 - acc: 0.9850 - val_loss: 0.9293 - val_acc: 0.8652
Epoch 11/20
157/157 - 18s - loss: 0.1119 - acc: 0.9900 - val_loss: 0.9557 - val_acc: 0.87

In [15]:
from keras.utils import plot_model

plot_model(model, to_file='model.png')

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
